# Keras
Librería para programar redes neuronales de una manera más sencilla que con TensorFlow. Keras se encuentra en una capa de abstracción por encima de TensorFlow.

[Documentación](https://keras.io/guides/)

In [ ]:
# !pip install tensorflow
# !pip install keras

Empezamos importando librerías

In [ ]:
# ============================================
# IMPORTACIÓN DE LIBRERÍAS NECESARIAS
# ============================================

# import tensorflow as tf  # TensorFlow es el framework base
from tensorflow import keras  # Keras es la API de alto nivel de TensorFlow
from tensorflow.keras import layers  # Módulo de capas para construir redes neuronales

import pandas as pd  # Para manipulación y análisis de datos
import numpy as np  # Para operaciones numéricas y manejo de arrays

Cargamos los datos de mnist. No vamos a tratar imagenes con redes convolucionales (perdemos la estructura espacial 2D). Todos los pixeles se convertirán en un vector de 28x28 features independientes, que serán las entradas del modelo.

In [ ]:
# ============================================
# CARGA DEL DATASET MNIST
# ============================================
# MNIST es un dataset clásico de dígitos escritos a mano (0-9)
# Cada imagen es de 28x28 píxeles en escala de grises

# Keras incluye datasets populares que podemos cargar directamente
# load_data() retorna 4 arrays: X_train, y_train, X_test, y_test
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

Vemos dimensiones del dataset

In [ ]:
# ============================================
# EXPLORACIÓN DE LAS DIMENSIONES DEL DATASET
# ============================================
'''
Resultado esperado:
- X_train: 60.000 imágenes de 28x28 píxeles (conjunto de entrenamiento)
- y_train: 60.000 etiquetas (los números del 0 al 9)
- X_test: 10.000 imágenes de prueba
- y_test: 10.000 etiquetas de prueba
'''

# Shape de los datos de entrenamiento
print(X_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)

# Shape de los datos de prueba
print(X_test.shape)   # (10000, 28, 28)
print(y_test.shape)   # (10000,)

60.000 imágenes de 28x28 pixeles. Vamos a representar una de ellas

In [ ]:
# ============================================
# VISUALIZACIÓN DE UNA IMAGEN DEL DATASET
# ============================================
import matplotlib.pyplot as plt

# Mostramos la primera imagen del conjunto de entrenamiento
# cmap='Greys' muestra la imagen en escala de grises
plt.imshow(X_train[0], cmap=plt.cm.get_cmap('Greys'));

Cada imagen se compone de 28x28 pixeles, y cada pixel representa una escala de grises que va del 0 al 255. Siendo 0 el blanco y 255 negro.

¿Se te ocurre alguna manera de normalizar los datos?

In [ ]:
# ============================================
# NORMALIZACIÓN DE LOS DATOS (FEATURE SCALING)
# ============================================
# Los valores de los píxeles van de 0 a 255
# Dividimos entre 255 para normalizar los valores entre 0 y 1
# Esto ayuda a que la red neuronal aprenda más rápido y mejor

# Convertimos a float32 para mayor precisión y dividimos por 255
X_train = X_train.astype("float32") / 255  # Normalización del conjunto de entrenamiento
X_test = X_test.astype("float32") / 255    # Normalización del conjunto de prueba

In [ ]:
# ============================================
# VERIFICACIÓN DE LA NORMALIZACIÓN
# ============================================
# Comprobamos que ahora los valores están entre 0 y 1
X_train[0]

In [ ]:
# ============================================
# CONVERSIÓN DE ETIQUETAS A FLOAT32
# ============================================
# Convertimos las etiquetas (y) a float32 por consistencia
y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

Guardamos datos para validación. Estos datos se usarán durante el entrenamiento. Otra opción es decirle a keras en la etapa de entrenamiento que reserve un X % de los datos para validar.

In [ ]:
# ============================================
# CREACIÓN DEL CONJUNTO DE VALIDACIÓN
# ============================================
# Separamos 10.000 muestras del conjunto de entrenamiento para validación
# La validación se usa durante el entrenamiento para evaluar el modelo
# en cada epoch y detectar overfitting

# Últimas 10.000 muestras para validación
X_val = X_train[-10000:]
y_val = y_train[-10000:]

# Las primeras 50.000 muestras quedan para entrenamiento
X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [ ]:
# ============================================
# VERIFICACIÓN DE LAS DIMENSIONES FINALES
# ============================================
print(X_train.shape)  # (50000, 28, 28) - Entrenamiento
print(X_val.shape)    # (10000, 28, 28) - Validación
print(X_test.shape)   # (10000, 28, 28) - Prueba

Montamos la arquitectura de la red neuronal. Se va a componer de:
* **Sequential**: API para iniciar la red neuronal. No cuenta como capa.
* **Flatten**: capa de entrada. Necesita un vector unidimensional. Como tenemos imágenes, esta capa aplana las imagenes (2D) en 1D.
* **Dense**: es una hidden layer. Se compondrá de `n` neuronas y de una función de activación que se aplicará a todas las neuronas de la capa.

Recuerda que es un problema de clasificación multiclase (10 clases) y que por tanto la última capa se compondrá de tantas neuronas como clases tengas.

En cuanto a las funciones de activación es recomendable usar relu en las hidden layer, que tarda menos en entrenar, mientras que la ultima (output) suele ser una softmax.

In [ ]:
# ============================================
# CONSTRUCCIÓN DE LA ARQUITECTURA DE LA RED NEURONAL
# ============================================
# Vamos a crear una red neuronal secuencial (capa por capa)

# Inicializamos el modelo secuencial
model = keras.models.Sequential()

# CAPA DE ENTRADA (Flatten)
# Convierte la matriz 2D (28x28) en un vector 1D de 784 elementos
# Esto es necesario porque las capas Dense requieren entrada 1D
model.add(keras.layers.Flatten(input_shape=(28, 28)))

# PRIMERA CAPA OCULTA (Hidden Layer 1)
# 300 neuronas con función de activación ReLU
# ReLU (Rectified Linear Unit): f(x) = max(0, x)
# Es rápida de calcular y evita el problema de gradientes que desaparecen
model.add(keras.layers.Dense(units=300,
                              activation='relu'))

# SEGUNDA CAPA OCULTA (Hidden Layer 2)
# 100 neuronas con función de activación ReLU
# Al tener menos neuronas, va reduciendo la dimensionalidad
model.add(keras.layers.Dense(units=100,
                              activation='relu'))

# CAPA DE SALIDA (Output Layer)
# 10 neuronas (una por cada dígito del 0 al 9)
# Softmax: convierte los valores en probabilidades que suman 1
# Ideal para problemas de clasificación multiclase
model.add(keras.layers.Dense(units=10,
                              activation='softmax'))

In [ ]:
# ============================================
# FORMA ALTERNATIVA DE DECLARAR LA RED NEURONAL
# ============================================
# Esta forma es más compacta y legible cuando tenemos muchas capas

# Definimos todas las capas en una lista
capas = [
    keras.layers.Flatten(input_shape=(28, 28)),  # Capa de entrada
    keras.layers.Dense(units=300, activation='relu'),  # Hidden layer 1
    keras.layers.Dense(units=100, activation='relu'),  # Hidden layer 2
    keras.layers.Dense(units=10, activation='softmax')  # Capa de salida
]

# Creamos el modelo pasando la lista de capas directamente
model = keras.models.Sequential(capas)

Podemos ver las capas, y acceder a sus elementos

In [ ]:
# ============================================
# INSPECCIÓN DE LAS CAPAS DEL MODELO
# ============================================
# Podemos acceder a cada capa individualmente usando índices
print(model.layers[0])  # Primera capa (Flatten)

Podemos ver los pesos de las capas sin entrenar, porque los inicializa aleatoriamente. Los bias los inicializa a 0.

In [ ]:
# ============================================
# VISUALIZACIÓN DE PESOS Y SESGOS INICIALES
# ============================================
# Keras inicializa los pesos aleatoriamente y los sesgos (bias) en 0
# Esto ocurre antes del entrenamiento

# Accedemos a la primera capa oculta (índice 1)
hidden1 = model.layers[1]

# get_weights() retorna una tupla: (weights, biases)
weights, biases = hidden1.get_weights()

In [ ]:
# Visualizamos la matriz de pesos (valores aleatorios iniciales)
weights

In [ ]:
# Número total de pesos en esta capa
# Debería ser: 784 (entradas) × 300 (neuronas) = 235,200
weights.size

Establecemos la configuración de ejecución... el compile.

In [ ]:
# ============================================
# COMPILACIÓN DEL MODELO (FORMA DETALLADA)
# ============================================
# La compilación configura el proceso de aprendizaje del modelo

model.compile(
    # OPTIMIZADOR: SGD (Stochastic Gradient Descent)
    # Actualiza los pesos usando el gradiente descendente estocástico
    optimizer=keras.optimizers.SGD(),
    
    # FUNCIÓN DE PÉRDIDA: Sparse Categorical Crossentropy
    # Mide el error entre las predicciones y las etiquetas reales
    # "Sparse" porque las etiquetas son enteros (0-9), no one-hot encoded
    loss=keras.losses.SparseCategoricalCrossentropy(),
    
    # MÉTRICA: Accuracy (precisión)
    # Porcentaje de predicciones correctas durante el entrenamiento
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
# ============================================
# COMPILACIÓN DEL MODELO (FORMA SIMPLIFICADA)
# ============================================
# Equivalente al anterior pero usando nombres de string
# Esta forma es más corta y comúnmente usada

model.compile(
    optimizer="sgd",  # Optimizador SGD
    loss="sparse_categorical_crossentropy",  # Función de pérdida
    metrics=["accuracy"]  # Métrica de evaluación
)

In [ ]:
# ============================================
# RESUMEN DEL MODELO
# ============================================
# Muestra la arquitectura completa: capas, shapes y parámetros
model.summary()

Entrenamos el modelo. Usamos los datos de entrenamiento. El batch_size es la cantidad de muestras que utiliza el SGD, y las epochs son las iteraciones que realiza en el entrenamiento.

In [ ]:
# Verificamos el shape de los datos de entrenamiento
X_train.shape  # (50000, 28, 28)

In [ ]:
# ============================================
# ENTRENAMIENTO DEL MODELO
# ============================================
# Aquí es donde la red neuronal "aprende" ajustando sus pesos

history = model.fit(
    X_train,  # Datos de entrada (imágenes)
    y_train,  # Etiquetas (targets)
    
    # BATCH_SIZE: número de muestras procesadas antes de actualizar pesos
    # Un batch más pequeño = más actualizaciones pero más lento
    batch_size=128,
    
    # EPOCHS: número de veces que el modelo ve todo el dataset
    # Más epochs = más aprendizaje, pero riesgo de overfitting
    epochs=50,
    
    # VALIDATION_DATA: datos para evaluar el modelo en cada epoch
    # Permite monitorear si hay overfitting (loss de validación aumenta)
    validation_data=(X_val, y_val)  # Alternativa: validation_split=0.1
)

Podemos reentrenar el modelo. No empieza de nuevo, sino que retoma el entrenamiento anterior.

In [ ]:
# ============================================
# REENTRENAMIENTO DEL MODELO
# ============================================
# Podemos seguir entrenando un modelo ya entrenado
# NO empieza desde cero, continúa desde donde se quedó
# Útil para hacer ajustes finos (fine-tuning)

model.fit(
    X_train,
    y_train,
    batch_size=64,  # Probamos con un batch diferente
    epochs=10,  # Solo 10 epochs adicionales
    validation_data=(X_val, y_val)
)

Veamos el histórico del entrenamiento, para poder representarlo posteriormente.

In [ ]:
# ============================================
# EXPLORACIÓN DEL HISTÓRICO DE ENTRENAMIENTO
# ============================================
# El objeto history contiene métricas de cada epoch
# print(history.params)  # Parámetros del entrenamiento
# print(history.epoch)   # Número de epochs ejecutadas
print(history.history)  # Diccionario con loss, accuracy, val_loss, val_accuracy

In [ ]:
# Visualizamos el diccionario completo del histórico
history.history

In [ ]:
# Vemos las métricas disponibles en el histórico
history.history.keys()  # dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
# Convertimos el histórico a DataFrame para visualizarlo mejor
pd.DataFrame(history.history)

In [ ]:
# ============================================
# VISUALIZACIÓN DEL PROCESO DE ENTRENAMIENTO
# ============================================
import pandas as pd
import matplotlib.pyplot as plt

# Graficamos todas las métricas (loss, accuracy, val_loss, val_accuracy)
pd.DataFrame(history.history).plot(figsize=(8, 5))

# Configuramos la gráfica
plt.grid(True)  # Añadimos cuadrícula
plt.gca().set_ylim(0, 1)  # Rango vertical de 0 a 1

# Interpretación:
# - Si val_loss aumenta mientras loss disminuye = OVERFITTING
# - Si ambas disminuyen = el modelo está aprendiendo bien
plt.show()

Si el modelo no ha ido bien, prueba a cambiar el learning rate, cambia de optimizador y después prueba a cambiar capas, neuronas y funciones de activación.

Ya tenemos el modelo entrenado. Probémoslo con test

In [ ]:
# ============================================
# EVALUACIÓN DEL MODELO CON DATOS DE PRUEBA
# ============================================
# Evaluamos el modelo con datos que NUNCA ha visto durante el entrenamiento
# Esto nos da una idea del rendimiento real del modelo

# evaluate() retorna [loss, accuracy]
results = model.evaluate(X_test, y_test)
results  # [pérdida en test, accuracy en test]

In [ ]:
# ============================================
# VISUALIZACIÓN DE UN EJEMPLO DE PRUEBA
# ============================================
# Mostramos la primera imagen del conjunto de prueba
plt.imshow(X_test[0].reshape(28, 28), cmap=plt.cm.get_cmap('Greys'));

In [ ]:
# Vemos los datos crudos de la primera imagen
X_test[:1]  # Array con valores normalizados entre 0 y 1

In [ ]:
# ============================================
# PREDICCIONES DEL MODELO
# ============================================
# Hacemos una predicción con la primera imagen del test

predictions = model.predict(X_test[:1])

# Shape de las predicciones: (1, 10)
# 1 muestra, 10 probabilidades (una por cada dígito 0-9)
print(predictions.shape)

# Redondeamos a 3 decimales para ver las probabilidades
# La suma de todas las probabilidades debe ser 1 (gracias a softmax)
np.round(predictions, 3)

In [ ]:
# ============================================
# OBTENER LA CLASE PREDICHA
# ============================================
# argmax() retorna el índice del valor más alto
# Es decir, el dígito con mayor probabilidad
predictions.argmax()  # Dígito predicho (0-9)

In [ ]:
# ============================================
# PREDICCIONES PARA TODO EL CONJUNTO DE PRUEBA
# ============================================
# Obtenemos las predicciones para todas las imágenes del test
# axis=1 indica que argmax se aplica a cada fila (cada predicción)
model.predict(X_test).argmax(axis=1)  # Array con todos los dígitos predichos

In [ ]:
# Visualizamos la segunda imagen del test
plt.imshow(X_test[1].reshape(28, 28), cmap=plt.cm.get_cmap('Greys'));

In [ ]:
# ============================================
# MATRIZ DE CONFUSIÓN
# ============================================
# Muestra los errores del modelo: qué dígitos confunde con cuáles
from sklearn.metrics import confusion_matrix, classification_report

# Filas = etiquetas reales, Columnas = predicciones
# Diagonal = predicciones correctas
confusion_matrix(y_test, model.predict(X_test).argmax(axis=1))

In [ ]:
# ============================================
# REPORTE DE CLASIFICACIÓN
# ============================================
# Muestra precision, recall, f1-score para cada clase (dígito)
# También incluye accuracy global y promedios macro/weighted
print(classification_report(y_test, model.predict(X_test).argmax(axis=1)))

### Problema de regresión
Veamos un ejemplo de cómo aplicar una red neuronal de TensorFlow a un problema de regresión.

In [ ]:
# ============================================
# CARGA DEL DATASET CALIFORNIA HOUSING
# ============================================
# Dataset de regresión: predecir precios de casas en California
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargamos el dataset
housing = fetch_california_housing()

# Convertimos a DataFrame para visualizar mejor
df = pd.DataFrame(housing.data, columns=housing.feature_names)
df['target'] = housing['target']  # Añadimos la columna objetivo (precio)

df.head()  # Primeras 5 filas

Divimos en train, test y validation

In [ ]:
# ============================================
# DIVISIÓN Y NORMALIZACIÓN DE DATOS
# ============================================

# PRIMERA DIVISIÓN: Train completo (75%) y Test (25%)
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data,
    housing.target
)

# SEGUNDA DIVISIÓN: Train (75% de 75%) y Validation (25% de 75%)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full,
    y_train_full
)

# ESTANDARIZACIÓN (StandardScaler)
# Transforma los datos para tener media=0 y desviación estándar=1
# IMPORTANTE: fit_transform solo en train, transform en valid/test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Aprende y transforma
X_valid = scaler.transform(X_valid)      # Solo transforma
X_test = scaler.transform(X_test)        # Solo transforma

In [ ]:
# Verificamos el shape de los datos de entrenamiento
# Debería mostrar (num_muestras, 8) porque hay 8 características
X_train.shape

Montamos el modelo. Simplemente se compondrá de una hidden layer, a la que le configuramos una capa previa de entrada de 8 neuronas (las features).

Se trata de un modelo de regresión, por lo que la capa de salida es una única neurona.

In [ ]:
# ============================================
# MODELO DE REGRESIÓN CON KERAS
# ============================================

# Creamos una red neuronal simple para regresión
model = keras.models.Sequential([
    # CAPA OCULTA
    # 30 neuronas con activación ReLU
    # input_shape=[8] porque tenemos 8 características
    # X_train.shape[1:] es otra forma de especificar las dimensiones
    keras.layers.Dense(30, activation='relu',
                       input_shape=X_train.shape[1:]),
    
    # CAPA DE SALIDA
    # 1 neurona SIN función de activación (regresión lineal)
    # Predice un valor continuo (precio de la casa)
    keras.layers.Dense(1)
])

# COMPILACIÓN
# loss='mean_squared_error': función de pérdida para regresión
# Mide la distancia cuadrática entre predicción y valor real
model.compile(loss="mean_squared_error",
              optimizer="sgd")

# ENTRENAMIENTO
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    validation_data=(X_valid, y_valid))

In [ ]:
# ============================================
# RESUMEN DEL MODELO DE REGRESIÓN
# ============================================
model.summary()
# Total params = (8 inputs × 30 neurons) + 30 bias + (30 × 1) + 1 bias = 271

In [ ]:
# ============================================
# EVALUACIÓN DEL MODELO DE REGRESIÓN
# ============================================
# Calculamos el MSE (Mean Squared Error) en el conjunto de test
mse_test = model.evaluate(X_test, y_test)
print(mse_test)  # Cuanto menor sea el MSE, mejor

In [ ]:
# ============================================
# PREDICCIONES DEL MODELO DE REGRESIÓN
# ============================================
# Predecimos los precios de las primeras 5 casas del test
y_pred = model.predict(X_test[:5])
y_pred  # Array con los precios predichos (valores continuos)

### Guardar modelo
Para guardar el modelo, en el formato de Keras (HDF5). 

In [ ]:
# ============================================
# GUARDAR EL MODELO
# ============================================
# Guardamos el modelo completo (arquitectura + pesos + configuración)
# Formato .keras es el recomendado (antes se usaba .h5)
model.save("my_keras_model.keras")

In [ ]:
# ============================================
# CARGAR UN MODELO GUARDADO
# ============================================
# Cargamos el modelo completo desde el archivo
# Podemos usar este modelo directamente para predicciones
model = keras.models.load_model("my_keras_model.keras")

### Callbacks
Son funciones predefinidas de Keras a aplicar durante el entrenamiento
Por ejemplo, `ModelCheckpoint` sirve para que el modelo se vaya guardando tras cada epoch. Así no perdemos el progreso en caso de que decidamos interrumpir el entrenamiento. El callback recibe como argumento el nombre del objeto donde queremos que se guarde el modelo entrenado.

In [ ]:
# ============================================
# CALLBACK: MODEL CHECKPOINT
# ============================================
# Guarda el modelo automáticamente después de cada epoch
# Útil para no perder progreso si se interrumpe el entrenamiento

checkpoint_cb = keras.callbacks.ModelCheckpoint("callback_model.h5")

# El modelo se guardará en "callback_model.h5" tras cada epoch
history = model.fit(X_train,
                    y_train,
                    epochs=30,
                    callbacks=[checkpoint_cb])

### Early Stopping
Interrumpe el entrenamiento cuando no ve progreso en el set de validación. Para ello tiene en cuenta un numero de epochs llamado `patience`. Se puede combinar con el callback

In [ ]:
# ============================================
# CALLBACK: EARLY STOPPING
# ============================================
# Detiene el entrenamiento automáticamente cuando no hay mejora
# Previene overfitting y ahorra tiempo de entrenamiento

# patience=3: espera 3 epochs sin mejora antes de detener
early_stopping_cb = keras.callbacks.EarlyStopping(patience=3)

# Podemos combinar múltiples callbacks en una lista
history = model.fit(X_train,
                    y_train,
                    epochs=50,  # Máximo 50 epochs
                    validation_data=(X_valid, y_valid),
                    # Se detendrá antes si val_loss no mejora durante 3 epochs
                    callbacks=[early_stopping_cb, checkpoint_cb])